In [10]:
import plotly.express as px
from bs4 import BeautifulSoup as bs 
import requests
import re
import numpy as np 
import pandas as pd
import camelot
import matplotlib.pyplot as plt 
import os



In [2]:
df

,country,continent,year,lifeExp,pop,gdpPercap,iso_alpha,iso_num
11,Afghanistan,Asia,2007,43.828,31889923,974.580338,AFG,4
23,Albania,Europe,2007,76.423,3600523,5937.029526,ALB,8
35,Algeria,Africa,2007,72.301,33333216,6223.367465,DZA,12
47,Angola,Africa,2007,42.731,12420476,4797.231267,AGO,24
59,Argentina,Americas,2007,75.320,40301927,12779.379640,ARG,32
...,...,...,...,...,...,...,...,...
1655,Vietnam,Asia,2007,74.249,85262356,2441.576404,VNM,704
1667,West Bank and Gaza,Asia,2007,73.422,4018332,3025.349798,PSE,275
1679,"Yemen, Rep.",Asia,2007,62.698,22211743,2280.769906,YEM,887
1691,Zambia,Africa,2007,42.384,11746035,1271.211593,ZMB,894


In [254]:
source = requests.get('https://www.who.int/emergencies/diseases/novel-coronavirus-2019/situation-reports').text
soup = bs(source, 'lxml')

hits = [x for x in soup.find_all('div', class_='sf-content-block content-block')
          if x.find(text=re.compile("Situation"))]
links = ['https://www.who.int'+i['href'] for i in hits[0].findAll('a', href=True)]

#fetch the pdf files
repFound = [(int)(file[6:-4]) for file in os.listdir('./') if file[:6] == 'report']
for i in links:
    num = re.split('-',i)[4]
    if (int)(num) not in repFound and (int)(num) > 29:
        with open('report'+num+'.pdf','wb') as f:
            f.write(requests.get(i, stream=True).content)

pdf = camelot.read_pdf('report44.pdf', pages='1-end')

In [269]:
#countries = [[country, lifeExp] for lifeExp, country in zip(df['lifeExp'],df['country'])]
df = px.data.gapminder().query("year==2007")
infected = []
for i in df['country']:
    x = ''
    for num,c in pdf[1].df[0].iteritems():
        if i == c: #match, insert
            x = pdf[1].df[1][num]
            break  
    for num,c in pdf[2].df[0].iteritems():
        if i == c: #match, insert
            x = pdf[2].df[1][num]
            break
    infected.append(x)

In [270]:


countries = pd.DataFrame({
    "countries": df['country'],
    "iso_alpha": df['iso_alpha'],
    'infected': infected
    
})

In [267]:
fig = px.choropleth(countries, locations="iso_alpha",
                    color="infected", # lifeExp is a column of gapminder
                    hover_name="countries", # column to add to hover information
                    color_continuous_scale=px.colors.sequential.Plasma)
fig.show()